## Demonstrations de la balistique

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import HTML

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead
output_notebook()

Loading BokehJS ...

#### Define global variables and function for the examples

In [2]:
g = 9.81 # g: gravity
omega = np.pi*25/180 # omega: angle for drawing the arrow on vectors

build_arrow_x = lambda v, a, x: np.linspace(v*np.cos(a) - 0.1*v*np.cos(a - x),v*np.cos(a), 10)
build_arrow_y = lambda v, a, x: np.linspace(v*np.sin(a) - 0.1*v*np.sin(a - x),v*np.sin(a), 10)

# Demo 1: trajectory of lauched bullet without including air resistance force

Below demonstrates the trajectory of the bullet without considering the air resistance force.

$\vec r = (tv_0\cos\alpha, -1/2gt^2+tv_0\sin\alpha)$

In [28]:

"""
Bokeh version -- without x_range the plot rescales the axis but 
if you don't pay attention to the fact that x becomes or order -8 
for alpha-> 90º then it may lead to confusions

TODO: 

DISPLAY: 
    * time to reach 0 again
    * max height
    * max length
    * any other?

"""

v_0_ini = 25
alpha_ini = np.pi*45/180

t = np.linspace(0,13,1000)
x = v_0_ini*np.cos(alpha_ini)*t
y = -0.5*g*t**2+v_0_ini*np.sin(alpha_ini)*t 

x2 = np.linspace(0, v_0_ini*np.cos(alpha_ini), 100)
y2 = np.linspace(0, v_0_ini*np.sin(alpha_ini), 100)

# Define plot
p = figure(title="Trajectory of the launched bullet", plot_height=360, plot_width=792, y_range=(0,135), x_range=(0,297),
           background_fill_color='#ffffff')

r = p.line(x, y, color="#b6e740", line_width=4.5, alpha=0.8) # r: trajectory of the bullet
v = p.line(x2, y2, color="#e30d4f", line_width=3, alpha=0.8) # v: velocity vector

# lines for drawing the arrow on velocity vector
a_up = p.line(build_arrow_x(v_0_ini, alpha_ini, omega), build_arrow_x(v_0_ini, alpha_ini, omega), color="#e30d4f", line_width=2, alpha=0.8)
a_down = p.line(build_arrow_x(v_0_ini, alpha_ini, -omega), build_arrow_x(v_0_ini, alpha_ini, -omega), color="#e30d4f", line_width=2, alpha=0.8)


p.xaxis.axis_label = 'Horizontal movement of the bullet'
p.yaxis.axis_label = 'Vertical movement of the bullet'

def update_bullet(v_0, alpha):
    alpharad=alpha*np.pi/180
    r.data_source.data['x'] = v_0*np.cos(alpharad)*t
    r.data_source.data['y'] = -0.5*g*t**2+v_0*np.sin(alpharad)*t 
    v.data_source.data['x'] = np.linspace(0, v_0*np.cos(alpharad), 100)
    v.data_source.data['y'] = np.linspace(0, v_0*np.sin(alpharad), 100)
    
    a_up.data_source.data['x'] = build_arrow_x(v_0, alpharad, omega)
    a_up.data_source.data['y'] = build_arrow_y(v_0, alpharad, omega)
    
    a_down.data_source.data['x'] = build_arrow_x(v_0, alpharad, -omega)
    a_down.data_source.data['y'] = build_arrow_y(v_0, alpharad, -omega)
    
    
    push_notebook()
    
show(p, notebook_handle=True)
interact(update_bullet, \
         v_0=widgets.IntSlider(min=0,max=50,step=1,value=25, description='$v_0$:'), \
         alpha=widgets.IntSlider(min=0,max=90,step=1,value=45, description='Alpha :'));




interactive(children=(IntSlider(value=25, description='$v_0$:', max=50), IntSlider(value=45, description='Alph…

# Demo 2: trajectory of lauched bullet including air resistance force

Below demonstrates the trajectory of the bullet when considering the air resistance force with adjustable $\vec v_0$, $\alpha$ and $\beta$ ($\beta$ is the proportionality between the air resistance coefficient and mass of bullet)

$\vec r= (\int_{0}^{t} v_0\cos\alpha e^{-\beta t} dt,\int_{0}^{t} \frac{-g}{\beta}+(v_0\sin\alpha+\frac{-g}{\beta}) e^{-\beta t}) dt $;


In [22]:

from scipy.integrate import cumtrapz


g = 9.81
v_0_ini = 25
alpha_ini = np.pi*45/180
beta = 0.5

t = np.linspace(0,15,1000)
x = v_0_ini*np.cos(alpha_ini)/beta * (1 - np.exp(-beta*t))
y = (g+beta*v_0_ini*np.sin(alpha_ini))*(1-np.exp(-beta*t))/(beta**2) - t*g/beta


x2 = np.linspace(0, v_0_ini*np.cos(alpha_ini), 100)
y2 = np.linspace(0, v_0_ini*np.sin(alpha_ini), 100)


p = figure(title="Trajectory of the launched bullet", plot_height=360, plot_width=792, y_range=(0,135), x_range=(0,297),
           background_fill_color='#ffffff')
r = p.line(x, y, color="#8888cc", line_width=4.5, alpha=0.8)
v = p.line(x2, y2, color="#e30d4f", line_width=3, alpha=0.8)

# lines for drawing the arrow on velocity vector
a_up = p.line(build_arrow_x(v_0_ini, alpha_ini, omega), build_arrow_x(v_0_ini, alpha_ini, omega), color="#e30d4f", line_width=2, alpha=0.8)
a_down = p.line(build_arrow_x(v_0_ini, alpha_ini, -omega), build_arrow_x(v_0_ini, alpha_ini, -omega), color="#e30d4f", line_width=2, alpha=0.8)


p.xaxis.axis_label = 'Horizontal movement of the bullet'
p.yaxis.axis_label = 'Verticle movement of the bullet'

def update_bullet(v_0, alpha, beta):
    alpharad=alpha*np.pi/180
    r.data_source.data['x'] = v_0*np.cos(alpharad)/beta * (1 - np.exp(-beta*t))
    r.data_source.data['y'] = (g+beta*v_0*np.sin(alpharad))*(1-np.exp(-beta*t))/(beta**2) - t*g/beta
    v.data_source.data['x'] = np.linspace(0, v_0*np.cos(alpharad), 100)
    v.data_source.data['y'] = np.linspace(0, v_0*np.sin(alpharad), 100)
    
    a_up.data_source.data['x'] = build_arrow_x(v_0, alpharad, omega)
    a_up.data_source.data['y'] = build_arrow_y(v_0, alpharad, omega)
    
    a_down.data_source.data['x'] = build_arrow_x(v_0, alpharad, -omega)
    a_down.data_source.data['y'] = build_arrow_y(v_0, alpharad, -omega)
    
    push_notebook()
    
show(p, notebook_handle=True)
interact(update_bullet, \
         v_0=widgets.IntSlider(min=0,max=50,step=1,value=25, description='$v_0$:'), \
         alpha=widgets.IntSlider(min=0,max=90,step=1,value=45, description='Alpha :'), \
        beta=widgets.FloatSlider(min=0.00001,max=1,step=0.001,value=0.5, description='Beta :'));




interactive(children=(IntSlider(value=25, description='$v_0$:', max=50), IntSlider(value=45, description='Alph…

# Demo 3: trajectory of lauched bullet with vs. without air resistance force

Below demonstrates the trajectory of the bullet with vs. without considering air resistance force

The trajectory of the bullet **without considering air resistance force** is given by:

$\vec r = (tv_0\cos\alpha, -1/2gt^2+tv_0\sin\alpha)$;


The derivation of the trajectory of the bullet **considering the air resistance force** is shown in the following lines: 


$\sum \vec F = m \vec a = \vec p + \vec F_{F} = -m g \vec e_{y} - b_{l} \vec v$

This equation in matrix form is represented as: 


$ \begin{pmatrix} m \ddot{x}\\  m \ddot{y} \end{pmatrix}  =  \begin{pmatrix} 0\\  -mg \end{pmatrix}  + 
\begin{pmatrix} -b_{l} \dot{x}\\  -b_{l} \dot{y} \end{pmatrix} $

The final system results in: 


$\left\{\begin{matrix} m \ddot{x} + b_{l}\dot{x} = 0\\  m \ddot{y} + b_{l}\dot{y} + mg  = 0\end{matrix}\right. $ 

By defining $\lambda = \frac{b_{l}}{m}$ we get:

$\left\{\begin{matrix} \ddot{x} + \lambda \dot{x} = 0\\  \ddot{y} + \lambda \dot{y} + g = 0 \end{matrix}\right. $

Trajectory of the bullet along x axis: 

$\frac{\mathrm{d} v_{x}}{\mathrm{d} t} + \lambda v_{x} = 0 \rightarrow \frac{\mathrm{d} v_{x}}{\mathrm{d} t} = -  \lambda v_{x} \rightarrow ln \left (  \frac{v_{x}}{v_{0_x}}  \right ) = - \lambda t$

$v_x = v_0 + cos \left ( \alpha \right )e^{-\lambda t} \rightarrow x\left ( t \right ) = \frac{v_{0}  cos\left ( \alpha \right )}{\lambda}\left [ 1 - e^{-\lambda t} \right ]$


Trajectory of the bullet along the y axis: 

$\frac{\mathrm{d} v_{y}}{\mathrm{d} t} = - \left ( g + \lambda v_{y} \right ) \rightarrow \frac{\mathrm{d} v_{y}}{g + \lambda v_{y}} = - \mathrm{d} t \rightarrow \frac{ 1}{ - \lambda } ln\left ( \frac{g + \lambda v_{y} }{g + \lambda v_{0_y} } \right ) = - t $

$g + \lambda v_{y} = \left (  g + \lambda v_{0} sin \left ( \alpha \right )\right ) e^{-\lambda t} \rightarrow 
v_y = \frac{g + \lambda v_{0} sin \left ( \alpha \right )}{\lambda} e^{-\lambda t} - \frac{g}{\lambda}$

$y\left ( t\right ) = - \frac{1}{\lambda } \frac{g + \lambda v_0sin\left ( \alpha \right )}{\lambda } \left [ e^{-\lambda t}  - 1 \right ] - \frac{g t}{\lambda} \rightarrow y\left ( t\right ) = \frac{g + \lambda v_0sin\left ( \alpha \right )}{\lambda^{2} } \left [ 1 - e^{-\lambda t}  \right ] - \frac{g t}{\lambda}$

Finally, the resulting trajectory vector is: 

$\vec r= (\frac{v_{0}  cos\left ( \alpha \right )}{\lambda}\left [ 1 - e^{-\lambda t} \right ], \frac{g + \lambda v_0sin\left ( \alpha \right )}{\lambda^{2} } \left [ 1 - e^{-\lambda t}  \right ] - \frac{g t}{\lambda}) $

In [23]:
v_0_ini = 25
alpha_ini = np.pi*45/180
beta = 0.5

t = np.linspace(0,15,1000)

x = v_0_ini*np.cos(alpha_ini)/beta * (1 - np.exp(-beta*t))
y = (g+beta*v_0_ini*np.sin(alpha_ini))*(1-np.exp(-beta*t))/(beta**2) - t*g/beta

x_no_f = v_0_ini*np.cos(alpha_ini)*t
y_no_f = -0.5*g*t**2+v_0_ini*np.sin(alpha_ini)*t 

x2 = np.linspace(0, v_0_ini*np.cos(alpha_ini), 100)
y2 = np.linspace(0, v_0_ini*np.sin(alpha_ini), 100)


p = figure(title="Trajectory of the launched bullet", plot_height=360, plot_width=792, y_range=(0,135), x_range=(0,297), 
           background_fill_color='#ffffff')
r = p.line(x, y, color="#8888cc", line_width=4.5, alpha=0.8, legend='With air resistance')
r_n =  p.line(x_no_f, y_no_f, color="#b6e740", line_width=4.5, alpha=0.8, legend='Without air resistance')
v = p.line(x2, y2, color="#e30d4f", line_width=3, alpha=0.8)



# lines for drawing the arrow on velocity vector
a_up = p.line(build_arrow_x(v_0_ini, alpha_ini, omega), build_arrow_x(v_0_ini, alpha_ini, omega), color="#e30d4f", line_width=2, alpha=0.8)
a_down = p.line(build_arrow_x(v_0_ini, alpha_ini, -omega), build_arrow_x(v_0_ini, alpha_ini, -omega), color="#e30d4f", line_width=2, alpha=0.8)


p.xaxis.axis_label = 'Horizontal movement of the bullet'
p.yaxis.axis_label = 'Verticle movement of the bullet'

def update_bullet(v_0, alpha, beta):
    alpharad=alpha*np.pi/180
    
    r.data_source.data['x'] = v_0*np.cos(alpharad)/beta * (1 - np.exp(-beta*t))
    r.data_source.data['y'] = (g+beta*v_0*np.sin(alpharad))*(1-np.exp(-beta*t))/(beta**2) - t*g/beta
    
    r_n.data_source.data['x'] = v_0*np.cos(alpharad)*t
    r_n.data_source.data['y'] = -0.5*g*t**2+v_0*np.sin(alpharad)*t
    
    v.data_source.data['x'] = np.linspace(0, v_0*np.cos(alpharad), 100)
    v.data_source.data['y'] = np.linspace(0, v_0*np.sin(alpharad), 100)
    
    a_up.data_source.data['x'] = build_arrow_x(v_0, alpharad, omega)
    a_up.data_source.data['y'] = build_arrow_y(v_0, alpharad, omega)
    
    a_down.data_source.data['x'] = build_arrow_x(v_0, alpharad, -omega)
    a_down.data_source.data['y'] = build_arrow_y(v_0, alpharad, -omega)
    
    push_notebook()
    
show(p, notebook_handle=True)
interact(update_bullet, \
         v_0=widgets.IntSlider(min=0,max=50,step=1,value=25, description='$v_0$:'), \
         alpha=widgets.IntSlider(min=0,max=90,step=1,value=45, description='Alpha :'), \
        beta=widgets.FloatSlider(min=0.00001,max=1,step=0.001,value=0.5, description='Beta :'));

interactive(children=(IntSlider(value=25, description='$v_0$:', max=50), IntSlider(value=45, description='Alph…

In [6]:
## CORRECT PLOT
#draw_arrow_v_x = lambda v, x, angle, omega: np.linspace((v+x), (v+x) - ) 10)
#draw_arrow_v_y = lambda v, x, angle, omega: np.linspace((v+x)*(1-0.1*np.sin(angle - omega )), v+x, 10)

v_0_ini = 15
alpha_ini = np.pi*45/180
beta = 0.5
point = 50

t = np.linspace(0,15,1000)

x = v_0_ini*np.cos(alpha_ini)/beta * (1 - np.exp(-beta*t))
y = (g+beta*v_0_ini*np.sin(alpha_ini))*(1-np.exp(-beta*t))/(beta**2) - t*g/beta

x2 = np.linspace(0, v_0_ini*np.cos(alpha_ini), 100)
y2 = np.linspace(0, v_0_ini*np.sin(alpha_ini), 100)


v_x_with = v_0_ini*np.cos(alpha_ini)*np.exp(-beta*t)
v_y_with = -g/beta+(v_0_ini*np.sin(alpha_ini)+g/beta)*np.exp(-beta*t)
v_with = np.sqrt((v_0_ini*np.cos(alpha_ini)*np.exp(-beta*t))**2+(-g/beta+(v_0_ini*np.sin(alpha_ini)+g/beta)*np.exp(-beta*t))**2)


p = figure(title="Details in a given point of the trajectory of a launched bullet with air resistance force", \
           plot_height=500, plot_width=500, y_range=(0,15), x_range=(0,15), background_fill_color='#ffffff', aspect_scale=1)

#Trajectory up to the point
r = p.line(x[0:point], y[0:point], color="#8888cc", line_width=4.5, alpha=0.8, legend='With air resistance')

# Initial velocity
v = p.line(x2, y2, color="#e30d4f", line_width=3, alpha=0.2, legend='Initial Velocity')

# lines for drawing the arrow on velocity vector
a_up = p.line(build_arrow_x(v_0_ini, alpha_ini, -omega), build_arrow_x(v_0_ini, alpha_ini, omega), color="#e30d4f", line_width=2, alpha=0.2, legend='Initial Velocity')
a_down = p.line(build_arrow_x(v_0_ini, alpha_ini, omega), build_arrow_x(v_0_ini, alpha_ini, -omega), color="#e30d4f", line_width=2, alpha=0.2, legend='Initial Velocity')

# Mark point in the trajectory
p.circle(x[point], y[point], size=5)


# Velocity at given point
v_p_x = np.linspace(x[point], x[point] + v_x_with[point], 10)
v_p_y = np.linspace(y[point], y[point] + v_y_with[point], 10)

v_p = p.line(v_p_x, v_p_y, color="#FFB749", line_width=3, legend='Velocity at given point')

angle_tg = v_y_with[point]/v_x_with[point]
angle_v = np.arctan(angle_tg)


a_down_v = p.line(np.linspace(v_x_with[point]+x[point], v_x_with[point]+x[point] -0.1*v_with[point]*np.cos(-angle_v + np.pi/4 + np.pi*20/180), 10),\
                np.linspace(v_y_with[point]+y[point], v_y_with[point]+y[point] +0.1*v_with[point]*np.sin(angle_v - np.pi/4 - np.pi*1/180), 10),\
                color="#FFB749", line_width=3, alpha=0.8, legend='Velocity at given point')

a_up_v = p.line(np.linspace(v_x_with[point]+x[point], v_x_with[point]+x[point] -0.1*v_with[point]*np.sin(angle_v + np.pi/4 + np.pi*20/180), 10),\
                np.linspace(v_y_with[point]+y[point], v_y_with[point]+y[point] +0.1*v_with[point]*np.cos(angle_v + np.pi/4 + np.pi*20/180), 10),\
                color="#FFB749", line_width=3, alpha=0.8, legend='Velocity at given point')


#a_up_v = p.line(draw_arrow_v_x(v_x_with[point], x[point], -angle_v, -np.pi*30/180 ), draw_arrow_v_y(v_y_with[point], y[point], -angle_v, np.pi*30/180), color="#1635CE", line_width=3, alpha=0.8)

f_p_x = np.linspace(x[point] - beta*v_x_with[point], x[point], 10)
f_p_y = np.linspace(y[point] -beta* v_y_with[point],y[point], 10)
f_p = p.line(f_p_x, f_p_y, color="#0e11ec", line_width=3, legend='Air resistance force')

a_up_f = p.line(np.linspace(x[point] - beta*v_x_with[point], \
                            x[point] - beta*v_x_with[point] - 0.1*v_with[point]*np.cos(angle_v - np.pi*(1-160)/180), 10), \
               np.linspace(y[point] - beta*v_y_with[point], \
                            y[point] - beta*v_y_with[point] - 0.1*v_with[point]*np.sin(angle_v - np.pi*(1-160)/180), 10), \
                            color='#0e11ec', line_width=3, legend='Air resistance force')

a_up_f = p.line(np.linspace(x[point] - beta*v_x_with[point], \
                            x[point] - beta*v_x_with[point] - 0.1*v_with[point]*np.cos(angle_v - np.pi*(1+160)/180), 10), \
               np.linspace(y[point] - beta*v_y_with[point], \
                            y[point] - beta*v_y_with[point] - 0.1*v_with[point]*np.sin(angle_v - np.pi*(1+160)/180), 10), \
                            color='#0e11ec', line_width=3,legend='Air resistance force')

fg_x = np.linspace(x[point], x[point], 10)
fg_y = np.linspace(y[point], 0.7*y[point], 10)
f_g = p.line(fg_x, fg_y, color="#0eaeec", line_width=2, legend='Gravity force')

a_up_g = p.line(np.linspace(x[point], x[point] + 0.3, 10), \
               np.linspace(0.7*y[point], 0.8*y[point], 10), color="#0eaeec", line_width=2, legend='Gravity force')
a_down_g = p.line(np.linspace(x[point], x[point] - 0.3, 10), \
               np.linspace(0.7*y[point], 0.8*y[point], 10), color="#0eaeec", line_width=2, legend='Gravity force')

p.legend.location = "top_left"
p.legend.click_policy="hide"


show(p)

# Demo 4: velocities of lauched bullet with vs. without air resistance force

Below demonstrates the velocity of the bullet with vs. without considering air resistance

$\vec v = (v_0\cos\alpha,-gt+v_0\sin\alpha)$; (without considering air resistance force)

$\vec v = (v_0\cos\alpha e^{-\beta t},\frac{-g}{\beta}+(v_0\sin\alpha+\frac{g}{\beta}) e^{-\beta t})$;(considering air resistance force)

In [24]:
g = 9.81

v_0 = 25
alpharad = np.pi*45/180
beta = 0.5

t = np.linspace(0,15,1000)

v_x_without = np.linspace(v_0*np.cos(alpharad),v_0*np.cos(alpharad), 1000)
v_y_without = v_0*np.sin(alpharad)-g*t
v_without = np.sqrt((v_0*np.cos(alpharad))**2+(v_0*np.sin(alpharad)-g*t)**2)
    
v_x_with = v_0*np.cos(alpharad)*np.exp(-beta*t)
v_y_with = -g/beta+(v_0*np.sin(alpharad)+g/beta)*np.exp(-beta*t)
v_with = np.sqrt((v_0*np.cos(alpharad)*np.exp(-beta*t))**2+(-g/beta+(v_0*np.sin(alpharad)+g/beta)*np.exp(-beta*t))**2)


p = figure(title="Velocity of the bullet", plot_height=500, plot_width=900, x_range=(0,16),
           background_fill_color='#ffffff')


vx1 = p.line(t, v_x_without, color="#360eea", line_width=4.5, alpha=0.8, \
             legend='Vx without considering air resistance force', line_dash='dashed')
vy1 = p.line(t, v_y_without, color="#e70f5c", line_width=4.5, alpha=0.8, \
             legend='Vy without considering air resistance force', line_dash='dashed')
v1 = p.line(t, v_without, color="#06e52b", line_width=4.5, alpha=0.8, \
             legend='V without considering air resistance force', line_dash='dashed')

vx2 = p.line(t, v_x_with, color="#360eea", line_width=4.5, alpha=0.8, \
             legend='Vx considers air resistance force')
vy2 = p.line(t, v_y_with, color="#e70f5c", line_width=4.5, alpha=0.8, \
             legend='Vy considers air resistance force')
v2 = p.line(t, v_with, color="#06e52b", line_width=4.5, alpha=0.8, \
             legend='V considers air resistance force')


p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Velocity'
p.legend.location = "bottom_left"
p.legend.click_policy="hide"

def update_bullet(v_0, alpha, beta):
    alpharad=alpha*np.pi/180
    
    vx1.data_source.data['y'] = np.linspace(v_0*np.cos(alpharad),v_0*np.cos(alpharad), 1000)
    vy1.data_source.data['y'] = v_0*np.sin(alpharad)-g*t
    v1.data_source.data['y'] = np.sqrt((v_0*np.cos(alpharad))**2+(v_0*np.sin(alpharad)-g*t)**2)
    
    vx2.data_source.data['y'] = v_0*np.cos(alpharad)*np.exp(-beta*t)
    vy2.data_source.data['y'] = -g/beta+(v_0*np.sin(alpharad)+g/beta)*np.exp(-beta*t)
    v2.data_source.data['y'] = np.sqrt((v_0*np.cos(alpharad)*np.exp(-beta*t))**2+(-g/beta+(v_0*np.sin(alpharad)+g/beta)*np.exp(-beta*t))**2)
    
    
    
    
    
    push_notebook()
    
show(p, notebook_handle=True)
interact(update_bullet, \
         v_0=widgets.IntSlider(min=0,max=50,step=1,value=25, description='$v_0$:'), \
         alpha=widgets.IntSlider(min=0,max=90,step=1,value=45, description='Alpha :'), \
        beta=widgets.FloatSlider(min=0.00001,max=1,step=0.001,value=0.5, description='Beta :'));

interactive(children=(IntSlider(value=25, description='$v_0$:', max=50), IntSlider(value=45, description='Alph…

# Demo 5: trajectories of lauched bullet on slopes

In [27]:
import warnings; warnings.simplefilter('ignore')

v_0 = 25
alpharad = np.pi*45/180
thetarad = np.pi*5/180
beta = 0.5

t = np.linspace(0,15,1000)

trajectory_x_without = v_0*np.cos(alpharad+thetarad)*t
trajectory_y_without = -0.5*g*t**2+v_0*np.sin(alpharad+thetarad)*t 
    
trajectory_x_with = v_0_ini*np.cos(alpharad+thetarad)/beta * (1 - np.exp(-beta*t))
trajectory_y_with = (g+beta*v_0_ini*np.sin(alpharad+thetarad))*(1-np.exp(-beta*t))/(beta**2) - t*g/beta
    
slope_height = np.linspace(0,270,10)*np.tan(thetarad)  


x2 = np.linspace(0, v_0_ini*np.cos(alpharad+thetarad), 10)
y2 = np.linspace(0, v_0_ini*np.sin(alpharad+thetarad), 10)


p = figure(title="Trajectory of the launched bullet", plot_height=360, plot_width=792, y_range=(0,135), x_range=(0,297), \
        background_fill_color='#ffffff')

r = p.line(trajectory_x_with, trajectory_y_with, color="#8888cc", \
           line_width=4.5, alpha=0.8, legend='With air resistance')

r_n =  p.line(trajectory_x_without, trajectory_y_without, color="#b6e740", \
              line_width=4.5, alpha=0.8, legend='Without air resistance', line_dash='dashed')

v = p.line(x2, y2, color="#e30d4f", line_width=3, alpha=0.8)

# lines for drawing the arrow on velocity vector
a_up = p.line(build_arrow_x(v_0_ini, alpha_ini+thetarad, omega), build_arrow_y(v_0_ini, alpha_ini+thetarad, omega), color="#e30d4f", line_width=2, alpha=0.8)
a_down = p.line(build_arrow_x(v_0_ini, alpha_ini+thetarad, -omega), build_arrow_y(v_0_ini, alpha_ini+thetarad, -omega), color="#e30d4f", line_width=2, alpha=0.8)


s = p.line(np.linspace(0,270,10), slope_height, color='firebrick', line_width=3, alpha=0.3)


p.xaxis.axis_label = 'Horizontal movement of the bullet'
p.yaxis.axis_label = 'Verticle movement of the bullet'

def update_bullet(v_0, alpha, beta, theta):
    alpharad=alpha*np.pi/180
    thetarad = theta*np.pi/180
    
    r_n.data_source.data['x'] = v_0*np.cos(alpharad+thetarad)*t
    r_n.data_source.data['y'] = -0.5*g*t**2+v_0*np.sin(alpharad+thetarad)*t 
    
    r.data_source.data['x'] = v_0*np.cos(alpharad+thetarad)/beta * (1 - np.exp(-beta*t))
    r.data_source.data['y'] = (g+beta*v_0*np.sin(alpharad+thetarad))*(1-np.exp(-beta*t))/(beta**2) - t*g/beta
    
    v.data_source.data['x'] = np.linspace(0, v_0*np.cos(alpharad+thetarad), 100)
    v.data_source.data['y'] = np.linspace(0, v_0*np.sin(alpharad+thetarad), 100)
    
    a_up.data_source.data['x'] = build_arrow_x(v_0, alpharad+thetarad, omega)
    a_up.data_source.data['y'] = build_arrow_y(v_0, alpharad+thetarad, omega)
    
    a_down.data_source.data['x'] = build_arrow_x(v_0, alpharad+thetarad, -omega)
    a_down.data_source.data['y'] = build_arrow_y(v_0, alpharad+thetarad, -omega)
    
    pos = np.where(r_n.data_source.data['y']>0)[0]
    s.data_source.data['y'] = s.data_source.data['x']*np.tan(thetarad)
    #s.data_source.data['x'] = r_n.data_source.data['x'][pos]
    
    
    if (alpha+theta)>90:
        print('Warning! Final angle greater than 90º')
    else:
        pass
        #p.circle(r_n.data_source.data['x'][idx], r_n.data_source.data['y'][idx], radius=1)
        #x.data_source.data['y'] =
            
    
    push_notebook()
    
show(p, notebook_handle=True);
interact(update_bullet, \
         v_0=widgets.IntSlider(min=0,max=50,step=1,value=25, description='$v_0$:'), \
         alpha=widgets.IntSlider(min=0,max=90,step=1,value=45, description='Alpha :'), \
        beta=widgets.FloatSlider(min=0.00001,max=1,step=0.001,value=0.5, description='Beta :'),\
         theta=widgets.FloatSlider(min=0,max=45,step=1,value=5, description='Theta :'));

interactive(children=(IntSlider(value=25, description='$v_0$:', max=50), IntSlider(value=45, description='Alph…